In [1]:
import os
from IPython.display import display,Markdown,Latex
from langchain.prompts import PromptTemplate
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import JsonOutputParser,StrOutputParser
from langchain_community.chat_models import ChatOllama
from langchain_community.tools import DuckDuckGoSearchRun
from langchain_community.utilities import DuckDuckGoSearchAPIWrapper
from langgraph.graph import END,StateGraph
from typing_extensions import TypedDict
from langchain_groq import ChatGroq

In [ ]:
# # Environment Variables
# os.environ['LANGCHAIN_TRACING_V2'] = 'true'
# os.environ["LANGCHAIN_PROJECT"] = "L3 Research Agent"

In [2]:
# # Defining LLM
local_llm = 'llama3:8b'
llama3 = ChatOllama(model=local_llm, temperature=0)
llama3_json = ChatOllama(model=local_llm, format='json', temperature=0)
# api_key = os.getenv("GROQ-API_KEY")
# llama3 = ChatGroq(model="llama-3.1-405b-reasoning", api_key=api_key,temperature=0.7, max_tokens=4096)
# llama3_json = ChatGroq(model="llama-3.1-405b-reasoning", api_key=api_key,temperature=0.7, max_tokens=4096,)

In [3]:
# Web Search Tool
# pip install -U duckduckgo_search==5.3.0b4
# ^ if running into 202 rate limit error

wrapper = DuckDuckGoSearchAPIWrapper(max_results=15)
web_search_tool = DuckDuckGoSearchRun(api_wrapper=wrapper)

# Test Run
resp = web_search_tool.invoke("home depot news")
resp

"Home Depot announced Thursday that it is spending $18.3 billion to buy SRS Distribution, a huge building-projects supplier that counts professional roofers, landscapers and pool contractors as its ... The Home Depot reports sales and earnings decline for the first quarter of fiscal 2024, but reaffirms its guidance for the year. The company also announces a pending acquisition of SRS Distribution Inc. and a conference call to discuss its performance. The Home Depot announced the expiration of the waiting period under the HSR Act for its pending acquisition of SRS, a leading distributor of professional tools and equipment. The transaction is expected to close on or about June 18, 2024, subject to customary closing conditions. The Home Depot is the world's largest home improvement specialty retailer. At the end of fiscal year 2023, the company operated a total of 2,335 retail stores in all 50 states, the District of Columbia , Puerto Rico , the U.S. Virgin Islands , Guam , 10 Canadian pr

In [4]:
# Generation Prompt

generate_prompt = PromptTemplate(
    template="""
    
    <|begin_of_text|>
    
    <|start_header_id|>system<|end_header_id|> 
    
    You are an AI assistant for Research Question Tasks, that synthesizes web search results. 
    Strictly use the following pieces of web search context to answer the question. If you don't know the answer, just say that you don't know. 
    keep the answer concise, but provide all of the details you can in the form of a research report. 
    Only make direct references to material if provided in the context.
    
    <|eot_id|>
    
    <|start_header_id|>user<|end_header_id|>
    
    Question: {question} 
    Web Search Context: {context} 
    Answer: 
    
    <|eot_id|>
    
    <|start_header_id|>assistant<|end_header_id|>""",
    input_variables=["question", "context"],
)

# Chain
generate_chain = generate_prompt | llama3 | StrOutputParser()

# Test Run
question = "How are you?"
context = ""
generation = generate_chain.invoke({"context": context, "question": question})
print(generation)

I'm just an AI, I don't have feelings or emotions like humans do. I am functioning properly and ready to assist with your research question tasks. I don't have personal experiences or physical sensations, so I can't say how I am in the classical sense. However, I am designed to provide accurate and helpful responses to your questions, and I'm always happy to help!


In [6]:
# Router

router_prompt = PromptTemplate(
    template="""
    
    <|begin_of_text|>
    
    <|start_header_id|>system<|end_header_id|>
    
    You are an expert at routing a user question to either the generation stage or web search. 
    Use the web search for questions that require more context for a better answer, or recent events.
    Otherwise, you can skip and go straight to the generation phase to respond.
    You do not need to be stringent with the keywords in the question related to these topics.
    Give a binary choice 'web_search' or 'generate' based on the question. 
    Return the JSON with a single key 'choice' with no premable or explanation. 
    
    Question to route: {question} 
    
    <|eot_id|>
    
    <|start_header_id|>assistant<|end_header_id|>
    
    """,
    input_variables=["question"],
)

# Chain
question_router = router_prompt | llama3_json | JsonOutputParser()

# Test Run
question = "What's up?"
print(question_router.invoke({"question": question}))

{'choice': 'generate'}


In [11]:
# Query Transformation

query_prompt = PromptTemplate(
    template="""
    
    <|begin_of_text|>
    
    <|start_header_id|>system<|end_header_id|> 
    
    You are an expert at crafting web search queries for research questions.
    More often than not, a user will ask a basic question that they wish to learn more about, however it might not be in the best format. 
    Reword their query to be the most effective web search string possible.
    Return the JSON with a single key 'query' with no premable or explanation. 
    
    Question to transform: {question} 
    
    <|eot_id|>
    
    <|start_header_id|>assistant<|end_header_id|>
    
    """,
    input_variables=["question"],
)

# Chain
query_chain = query_prompt | llama3_json | JsonOutputParser()

# Test Run
# question = "What's happened recently with Macom?"
# print(query_chain.invoke({"question": question}))

In [12]:
# Graph State
class GraphState(TypedDict):
    """
    Represents the state of our graph.

    Attributes:
        question: question
        generation: LLM generation
        search_query: revised question for web search
        context: web_search result
    """
    question : str
    generation : str
    search_query : str
    context : str

# Node - Generate

def generate(state):
    """
    Generate answer

    Args:
        state (dict): The current graph state

    Returns:
        state (dict): New key added to state, generation, that contains LLM generation
    """
    
    print("Step: Generating Final Response")
    question = state["question"]
    context = state["context"]

    # Answer Generation
    generation = generate_chain.invoke({"context": context, "question": question})
    return {"generation": generation}

# Node - Query Transformation

def transform_query(state):
    """
    Transform user question to web search

    Args:
        state (dict): The current graph state

    Returns:
        state (dict): Appended search query
    """
    
    print("Step: Optimizing Query for Web Search")
    question = state['question']
    gen_query = query_chain.invoke({"question": question})
    search_query = gen_query["query"]
    return {"search_query": search_query}


# Node - Web Search"
def web_search(state):
    """
    Web search based on the question

    Args:
        state (dict): The current graph state

    Returns:
        state (dict): Appended web results to context
    """

    search_query = state['search_query']
    print(f'Step: Searching the Web for: "{search_query}"')
    
    # Web search tool call
    search_result = web_search_tool.invoke(search_query)
    return {"context": search_result}


# Conditional Edge, Routing

def route_question(state):
    """
    route question to web search or generation.

    Args:
        state (dict): The current graph state

    Returns:
        str: Next node to call
    """

    print("Step: Routing Query")
    question = state['question']
    output = question_router.invoke({"question": question})
    if output['choice'] == "web_search":
        print("Step: Routing Query to Web Search")
        return "websearch"
    elif output['choice'] == 'generate':
        print("Step: Routing Query to Generation")
        return "generate"

In [13]:
# Build the nodes
workflow = StateGraph(GraphState)
workflow.add_node("websearch", web_search)
workflow.add_node("transform_query", transform_query)
workflow.add_node("generate", generate)

# Build the edges
workflow.set_conditional_entry_point(
    route_question,
    {
        "websearch": "transform_query",
        "generate": "generate",
    },
)
workflow.add_edge("transform_query", "websearch")
workflow.add_edge("websearch", "generate")
workflow.add_edge("generate", END)

# Compile the workflow
local_agent = workflow.compile()

In [14]:
from langsmith import traceable

# # Environment Variables
os.environ['LANGCHAIN_API_KEY'] = ''
os.environ['LANGCHAIN_TRACING_V2'] = 'true'
os.environ["LANGCHAIN_PROJECT"] = "L3 Research Agent"

@traceable 
def run_agent(query):
    output = local_agent.invoke({"question": query})
    print("=======")
    display(Markdown(output["generation"]))

In [15]:
# Test it out!
run_agent("What's are Apple's q3 earnings")

Step: Routing Query
Step: Routing Query to Web Search
Step: Optimizing Query for Web Search
Step: Searching the Web for: "Apple Q3 earnings report"
Step: Generating Final Response


Based on the provided web search context, Apple's Q3 earnings for fiscal year 2023 are:

* Revenue: $81.8 billion, down 1% year over year
* Earnings per diluted share: $1.26, up 5% year over year

These results were announced on August 3, 2023, and can be found in the earnings call transcript provided by Apple Inc.

In [16]:
from langsmith import Client

client = Client()

examples = [
    ("What Apple's Q3 Earnings?", "Apple today announced financial results for its fiscal 2023 third quarter ended July 1, 2023. The Company posted quarterly revenue of $81.8 billion, down 1 percent year over year, and quarterly earnings per diluted share of $1.26, up 5 percent year over year."),
    ("What are new apple products?", "Apple is refreshing both iPad Pro models with OLED screens, bringing a major update in display quality. There will be two models with screen sizes around 11 and 13 inches, and we are expecting design updates. With the switch to OLED, Apple is cutting down on thickness, and the new iPad Pro models will be much thinner. We're also expecting them to adopt the M3 chip for faster performance, and Apple is planning to debut a new Magic Keyboard that gives the iPad Pro a more Mac-like feel and a new Apple Pencil.  With the 2024 iPad Air refresh, we're getting two models for the first time. The smaller iPad Air will have a 10.9-inch display like the current iPad Air, while the larger version will have a 12.9-inch display like the current 12.9-inch iPad Pro. The iPad Air models will be more affordable than the iPad Pro models, and won't have \"Pro\" features like ProMotion refresh rates and OLED displays. Rumors are mixed on whether the iPad Air will get the M2 or the M3 chip, but either option will be an improvement over the M1 in the current model."),
]

dataset_name = "Apple - L3 Agent Testing"
if not client.has_dataset(dataset_name=dataset_name):
    dataset = client.create_dataset(dataset_name=dataset_name)
    inputs, outputs = zip(
        *[({"input": input}, {"expected": expected}) for input, expected in examples]
    )
    client.create_examples(inputs=inputs, outputs=outputs, dataset_id=dataset.id)

In [23]:
from langsmith.evaluation import LangChainStringEvaluator, evaluate
from langsmith.schemas import Example, Run
# from langchain_openai import ChatOpenAI
from langchain_groq import ChatGroq
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.pydantic_v1 import BaseModel, Field

# Search Tool Test
def search_retrieval(root_run: Run, example: Example) -> dict:
    """
    A simple evaluator that checks if the retrieved web search contains answer for the question
    """
    # Get documents and answer
    agent_run = next(run for run in root_run.child_runs if run.name == "run_agent")
    LangGraph = next(run for run in agent_run.child_runs if run.name == "LangGraph")
    search_run = next(run for run in LangGraph.child_runs if run.name == "websearch")
    context = search_run.outputs["context"]
    question = agent_run.inputs["query"]

    # Data model
    class GradeWebsearch(BaseModel):
        """Binary score for whether websearch contains question context."""

        binary_score: int = Field(description="Context contains answer to question, 1 or 0")

    # LLM with function call
    api_key = os.getenv("GROQ-API_KEY")
    llm = ChatGroq(model="llama3-70b-8192", api_key=api_key,temperature=0.7, max_tokens=2000)
    structured_websearch_grader = llm.with_structured_output(GradeWebsearch)

    # Prompt
    system = """You are a grader assessing whether an Web search contains the context needed to answer a user query. \n
        Give a binary score 1 or 0, where 1 means that the answer is in the web search results."""
    websearch_prompt = ChatPromptTemplate.from_messages(
        [
            ("system", system),
            ("human", "Web search: \n\n {context} \n\n Question: {question}"),
        ]
    )

    websearch_grader = websearch_prompt | structured_websearch_grader
    score = websearch_grader.invoke({"context": context, "question": question})
    return {"key": "websearch_verification", "score": int(score.binary_score)}

# Hallucination Test
def hallucination(root_run: Run, example: Example) -> dict:
    """
    A simple evaluator that checks to see the answer is grounded in the context
    """
    # Get documents and answer
    agent_run = next(run for run in root_run.child_runs if run.name == "run_agent")
    LangGraph = next(run for run in agent_run.child_runs if run.name == "LangGraph")
    search_run = next(run for run in LangGraph.child_runs if run.name == "websearch")
    context = search_run.outputs["context"]
    generation = LangGraph.outputs["generation"]

    # Data model
    class GradeHallucinations(BaseModel):
        """Binary score for hallucination present in generation answer."""

        binary_score: int = Field(description="Answer is grounded in the facts, 1 or 0")

    # LLM with function call
    api_key = os.getenv("GROQ-API_KEY")
    llm = ChatGroq(model="llama3-70b-8192", api_key=api_key,temperature=0.7, max_tokens=2000)
    structured_llm_grader = llm.with_structured_output(GradeHallucinations)

    # Prompt
    system = """You are a grader assessing whether an LLM generation is grounded in / supported by a set of retrieved facts. \n
        Give a binary score 1 or 0, where 1 means that the answer is grounded in / supported by the set of facts."""
    hallucination_prompt = ChatPromptTemplate.from_messages(
        [
            ("system", system),
            ("human", "Set of facts: \n\n {context} \n\n LLM generation: {generation}"),
        ]
    )

    hallucination_grader = hallucination_prompt | structured_llm_grader
    score = hallucination_grader.invoke({"context": context, "generation": generation})
    return {"key": "answer_hallucination", "score": int(score.binary_score)}

In [24]:
experiment_results = evaluate(
    lambda inputs: run_agent(inputs["input"]),
    data="Apple - L3 Agent Testing",
    evaluators=[search_retrieval, hallucination],
    experiment_prefix="websearch-test-1"
)

View the evaluation results for experiment: 'websearch-test-1-8f611243' at:
https://smith.langchain.com/o/b0942edf-d327-5af3-a8cb-6cfd8fb13d55/datasets/db0a88ba-7e1b-473f-9cdb-ca21cab4e41f/compare?selectedSessions=0e32113a-ee32-42ff-80cb-bbd8b518bcc9




0it [00:00, ?it/s]

Step: Routing Query
Step: Routing Query
Step: Routing Query to Web Search
Step: Optimizing Query for Web Search
Step: Routing Query to Web Search
Step: Optimizing Query for Web Search
Step: Searching the Web for: "Apple Q3 earnings report"
Step: Generating Final Response
Step: Searching the Web for: "What's new from Apple? OR (Apple latest products) OR (new Apple devices)"
Step: Generating Final Response


Based on the provided web search context, some of the new Apple products mentioned include:

1. iPad Pro with M4 processor: This is a significant upgrade to the existing iPad Pro models, offering 4 times the performance.
2. Apple Pencil Pro: A new version of the Apple Pencil designed for the iPad Air and iPad Pro.
3. Magic Keyboard: A new keyboard designed for the iPad Air and iPad Pro.

Additionally, there are rumors about upcoming products, including:

1. Low-cost version of AirPods: Analyst Ming-Chi Kuo has reported that Apple is working on a low-cost version of AirPods, which may be released in the future.
2. New Apple TV: There have been rumors that Apple could launch a new Apple TV in the first half of 2024.

It's worth noting that these are just rumors and not officially confirmed by Apple.

Based on the provided web search context, here is a concise research report on Apple's Q3 earnings:

**Fiscal 2023 Third Quarter Earnings**

* Revenue: $81.8 billion (down 1% year over year)
* Earnings per diluted share: $1.26 (up 5% year over year)

**Key Takeaways**

* Apple reported an all-time revenue record, despite a 1% YoY drop
* iPhone sales missed analysts' expectations
* Revenue fell 1.4% compared to the same quarter last year

**Upcoming Earnings Report**

* Apple will report its Q3 2024 earnings on Thursday, August 1
* The report will cover the months of April, May, and June, including the launch of the new M3 chip

**Past Earnings Reports**

* Apple reported $90.8 billion in revenue for the March quarter (Q2 2024), down 4% year over year
* Quarterly earnings per diluted share were $1.53

Note: The provided context does not mention any specific product categories or business updates beyond the general revenue and earnings figures.